In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/05/04 21:08:17 WARN Utils: Your hostname, MacBook-Pro-Sofi.local resolves to a loopback address: 127.0.0.1; using 109.123.185.23 instead (on interface en6)
23/05/04 21:08:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 21:08:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/04 21:08:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.csv('Datasets/DS_2019_public.csv', sep=',', header=True, inferSchema=True)

In [5]:
target_col = 'Climate_Region_Pub'

selected_cols = [
        'KWH', 'KWHCOL', 'KWHSPH', 'KWHWTH', 'KWHRFG', 'TOTSQFT', 'TOTSQFT_EN', 'TOTHSQFT', 'TOTCSQFT', 'TOTALBTU', 'TOTALBTUSPH', 'TOTALBTUCOL', 'TOTALBTUWTH', 'NOTMOIST', 'BTUEL', 'BTUELSPH', 'BTUELCOL', 
    ]

label_col = 'Marine_Region'

bad_values = [0.0]


In [6]:
for i in bad_values:
    expr = ' and '.join(f'(%s != {i})' % c for c in selected_cols)
    data = data.filter(expr)
data.show()

23/05/04 21:08:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+--------+-----------------+--------+-----------+--------+---------+-----------+-----------+-----------+------+-----------+--------+--------+-----+--------+--------+-----------+---------+--------+--------+--------+---------+-------+------+-----------+--------+-----------+-----+--------+--------+--------+--------+---------+------+-----+--------+--------+--------+--------+--------+--------+-----+-----------+--------+--------+-------+-----------+--------+--------+------------+---------+---------+---------+-------+----------+--------+--------+-------+--------+--------+--------+---------+------+-------+-------+--------+--------+--------+---------+--------+--------+--------+---------+--------+-------+------------+---------+-----------+-----------+--------+--------+----+------+-------+--------+-----+----+-------+-------+-----------+--------+-------+--------+-----------+---------+---------+---------+-----------+--------+-----+--------+---------+-----+------+--------+--------

In [7]:
data = data.withColumn(label_col, (col(target_col) == 5).cast('integer'))
data.show()

+------------------+--------+-----------------+--------+-----------+--------+---------+-----------+-----------+-----------+------+-----------+--------+--------+-----+--------+--------+-----------+---------+--------+--------+--------+---------+-------+------+-----------+--------+-----------+-----+--------+--------+--------+--------+---------+------+-----+--------+--------+--------+--------+--------+--------+-----+-----------+--------+--------+-------+-----------+--------+--------+------------+---------+---------+---------+-------+----------+--------+--------+-------+--------+--------+--------+---------+------+-------+-------+--------+--------+--------+---------+--------+--------+--------+---------+--------+-------+------------+---------+-----------+-----------+--------+--------+----+------+-------+--------+-----+----+-------+-------+-----------+--------+-------+--------+-----------+---------+---------+---------+-----------+--------+-----+--------+---------+-----+------+--------+--------

In [8]:
assembler = VectorAssembler(inputCols=selected_cols, outputCol='features')
model = assembler.transform(data).select('features', label_col)
model.show()

+--------------------+-------------+
|            features|Marine_Region|
+--------------------+-------------+
|[11688.0,33.268,3...|            0|
|[8490.0,2684.768,...|            0|
|[14943.0,4083.348...|            0|
|[19050.0,1478.869...|            0|
|[16357.0,2302.671...|            0|
|[25811.0,697.477,...|            0|
|[19344.0,2665.451...|            0|
|[15459.0,347.756,...|            0|
|[24563.0,1792.879...|            0|
|[22822.0,434.473,...|            0|
|[8504.0,1052.498,...|            0|
|[20967.0,316.808,...|            0|
|[1891.0,216.766,6...|            0|
|[33223.0,1157.343...|            0|
|[17093.0,499.266,...|            0|
|[24115.0,3185.955...|            0|
|[17167.0,208.186,...|            0|
|[9792.0,50.44,971...|            0|
|[25800.0,478.937,...|            0|
|[23630.0,3702.416...|            0|
+--------------------+-------------+
only showing top 20 rows



In [9]:
train_data, test_data = model.randomSplit([0.7, 0.3], seed = 20)

In [10]:
logistic_regression = LogisticRegression(labelCol=label_col).fit(train_data)

23/05/04 21:08:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/04 21:08:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [11]:
train_predictions = logistic_regression.evaluate(train_data).predictions
test_predictions = logistic_regression.evaluate(test_data).predictions
test_predictions.show()

/Users/sofi/anaconda3/envs/pyspark_env/lib/python3.10/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-------------+--------------------+--------------------+----------+
|            features|Marine_Region|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|[3679.0,179.132,1...|            0|[6.81464634195441...|[0.99890362206791...|       0.0|
|[5939.0,133.233,1...|            0|[20.5066354271837...|[0.99999999875811...|       0.0|
|[6739.0,102.844,1...|            0|[10.0331061426189...|[0.99995608040856...|       0.0|
|[6759.0,1302.381,...|            0|[10.0002294088951...|[0.99995461254430...|       0.0|
|[8504.0,1052.498,...|            0|[9.58052084992639...|[0.99993094380113...|       0.0|
|[8619.0,452.081,2...|            0|[5.12433215078651...|[0.99408500548098...|       0.0|
|[9077.0,139.101,7...|            0|[29.9443369727939...|[0.99999999999990...|       0.0|
|[9792.0,50.44,971...|            0|[40.4863669938955...|           [1.0,0.0]|       0.0|
|[9825.0,4

In [12]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol=label_col)
train_accuracy = evaluator.evaluate(train_predictions)
test_accuracy = evaluator.evaluate(test_predictions)
print(f'Train accuracy: {train_accuracy}')
print(f'Test accuracy: {test_accuracy}')

Train accuracy: 0.875
Test accuracy: 0.9921875
